In [2]:
# Install PT2.0,#
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/test/rocm5.6/

# Install timm
!pip3 install timm viztracer tabulate ipywidgets

# Imports
import torch 
import timm
import tabulate
import torch._dynamo
import logging
import transformers
from typing import List
from viztracer import VizTracer


Looking in indexes: https://download.pytorch.org/whl/test/rocm5.6/


## Example - resnet18

In [7]:
torch._logging.set_logs()

torch._dynamo.reset()

model = timm.create_model('resnet18', pretrained=False, num_classes=2).to('cuda:0')

# Wraps the original torch.nn.Module and later patches to optimised self.forward method
opt_model = torch.compile(model, backend="inductor")

print(type(model))
print(type(opt_model))

print(model)
print(opt_model)

<class 'timm.models.resnet.ResNet'>
<class 'torch._dynamo.eval_frame.OptimizedModule'>
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3

In [8]:
torch._logging.set_logs()
torch._dynamo.reset()

input_batch = torch.randn(64,3,7,7).to('cuda:0')

model = timm.create_model('resnet18', pretrained=False, num_classes=2).to('cuda:0')
opt_model = torch.compile(model, backend="inductor")

# Wraps the original torch.nn.Module and later patches to optimised self.forward method
opt_model(input_batch)

tensor([[ 0.2383, -0.1939],
        [ 0.6572,  0.1961],
        [-0.1838,  0.0419],
        [ 0.1979, -0.5030],
        [ 0.0690, -0.4075],
        [-0.1802,  0.5213],
        [ 0.7444, -0.8149],
        [ 0.6839, -0.0070],
        [-0.0533, -0.0607],
        [ 0.0489,  0.2604],
        [ 0.4726, -0.5212],
        [ 0.9182, -0.3004],
        [ 0.1978,  0.1747],
        [-0.1231, -0.0444],
        [-0.2041, -0.1494],
        [-0.2494, -0.3497],
        [ 0.0855, -0.8035],
        [ 0.2432,  0.1111],
        [ 0.3039, -0.1925],
        [-0.2229, -0.6068],
        [ 0.1484,  0.1843],
        [ 0.3748, -0.7154],
        [ 0.1294,  0.0347],
        [-0.1229, -0.1495],
        [ 0.0286, -0.1407],
        [-0.1305, -0.6298],
        [ 0.4487, -0.0850],
        [ 0.6505, -0.0454],
        [ 0.3222, -0.1426],
        [-0.0153, -0.3810],
        [ 1.4025, -0.6599],
        [-0.0867,  0.2649],
        [-0.1993,  0.5970],
        [ 0.6469, -0.3114],
        [ 0.5813, -0.1107],
        [ 0.5300, -0

In [40]:
torch._dynamo.reset()

with torch.profiler.profile() as prof:
    with torch.profiler.record_function("torch.compile call"):
        input_batch = torch.randn(64,3,7,7).to('cuda:0')
        opt_model = torch.compile(model, backend="inductor")
    with torch.profiler.record_function("opt_model(input_batch)"):
        opt_model(input_batch)

prof.export_chrome_trace("trace.json")

STAGE:2023-09-06 23:01:34 19834:19834 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-09-06 23:01:40 19834:19834 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-09-06 23:01:40 19834:19834 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


### Dynamo output

In [41]:
# Dynamo tracing
torch._logging.set_logs(trace_source=True)
torch._dynamo.reset()
input_batch = torch.randn(64,3,7,7).to('cuda:0')
opt_model(input_batch)

[2023-09-06 23:06:57,100] [28/0] torch._dynamo.symbolic_convert.__trace_source: [DEBUG] TRACE starts_line forward /usr/local/lib/python3.8/dist-packages/timm/models/resnet.py:540
[2023-09-06 23:06:57,100] [28/0] torch._dynamo.symbolic_convert.__trace_source: [DEBUG]         def forward(self, x):
[2023-09-06 23:06:57,104] [28/0] torch._dynamo.symbolic_convert.__trace_source: [DEBUG] TRACE starts_line forward /usr/local/lib/python3.8/dist-packages/timm/models/resnet.py:541
[2023-09-06 23:06:57,104] [28/0] torch._dynamo.symbolic_convert.__trace_source: [DEBUG]             x = self.forward_features(x)
[2023-09-06 23:06:57,105] [28/0] torch._dynamo.symbolic_convert.__trace_source: [DEBUG] TRACE starts_line forward_features /usr/local/lib/python3.8/dist-packages/timm/models/resnet.py:519 (inline depth: 1)
[2023-09-06 23:06:57,105] [28/0] torch._dynamo.symbolic_convert.__trace_source: [DEBUG]         def forward_features(self, x):
[2023-09-06 23:06:57,105] [28/0] torch._dynamo.symbolic_conver

tensor([[-0.6776, -0.7374],
        [-0.8101, -0.0034],
        [ 0.1255, -0.2349],
        [-0.4029, -0.1299],
        [ 0.1070, -0.3228],
        [-0.5940, -0.0554],
        [-0.4449, -0.9274],
        [ 0.0860, -0.1852],
        [ 0.1218, -0.5544],
        [-1.3032,  0.0221],
        [-0.3760, -0.4635],
        [-0.2732, -0.5201],
        [ 0.2781, -0.3265],
        [-0.0740, -0.2682],
        [-0.5682, -1.0316],
        [-0.2384, -0.3366],
        [-0.3340, -0.0929],
        [-0.2389, -0.2243],
        [-0.7796, -0.2437],
        [-0.0260, -0.1377],
        [ 0.0033, -0.0315],
        [ 0.2189, -0.1909],
        [-0.5649, -0.0356],
        [-0.0759, -0.3170],
        [ 0.0263, -0.8109],
        [-0.0387, -0.9966],
        [-0.8074, -1.0785],
        [ 0.1519, -0.3504],
        [-0.6180, -0.7323],
        [-0.2960, -0.6540],
        [ 0.3075, -0.7503],
        [ 0.0692, -0.3588],
        [-0.1934,  0.4809],
        [-0.4050, -0.5018],
        [ 0.3792, -0.0941],
        [-0.0811, -0

In [4]:
# Bytecode transformation
torch._logging.set_logs(bytecode=True)
torch._dynamo.reset()
input_batch = torch.randn(64,3,7,7).to('cuda:0')
opt_model(input_batch)

[2023-09-07 14:55:01,137] [1/0] torch._dynamo.convert_frame.__bytecode: [DEBUG] ORIGINAL BYTECODE forward /usr/local/lib/python3.8/dist-packages/timm/models/resnet.py line 540 
[2023-09-07 14:55:01,137] [1/0] torch._dynamo.convert_frame.__bytecode: [DEBUG] 541           0 LOAD_FAST                0 (self)
[2023-09-07 14:55:01,137] [1/0] torch._dynamo.convert_frame.__bytecode: [DEBUG]               2 LOAD_METHOD              0 (forward_features)
[2023-09-07 14:55:01,137] [1/0] torch._dynamo.convert_frame.__bytecode: [DEBUG]               4 LOAD_FAST                1 (x)
[2023-09-07 14:55:01,137] [1/0] torch._dynamo.convert_frame.__bytecode: [DEBUG]               6 CALL_METHOD              1
[2023-09-07 14:55:01,137] [1/0] torch._dynamo.convert_frame.__bytecode: [DEBUG]               8 STORE_FAST               1 (x)
[2023-09-07 14:55:01,137] [1/0] torch._dynamo.convert_frame.__bytecode: [DEBUG] 
[2023-09-07 14:55:01,137] [1/0] torch._dynamo.convert_frame.__bytecode: [DEBUG] 542          

tensor([[ 5.7302e-01,  1.6953e-01],
        [ 3.1415e-01, -1.9879e-01],
        [ 2.2843e-01, -5.1540e-02],
        [ 1.3476e-01, -5.3632e-01],
        [ 2.5758e-01,  3.6784e-01],
        [-2.0255e-01,  2.4212e-01],
        [-7.0895e-01,  4.0644e-02],
        [ 8.6189e-01, -1.9660e-01],
        [ 1.3331e-01, -2.1293e-01],
        [ 3.6175e-01,  1.1752e-01],
        [ 8.8030e-02, -7.1310e-01],
        [ 7.2991e-02,  1.0708e-01],
        [-1.9656e-02, -4.7510e-01],
        [-2.2432e-03,  2.8744e-01],
        [ 3.5546e-02, -4.8751e-01],
        [-5.6303e-01, -1.1960e-01],
        [ 5.1516e-01,  6.0561e-01],
        [-4.4132e-01,  1.2456e-01],
        [ 3.2252e-01,  1.1029e-03],
        [ 4.3625e-01, -1.1928e-01],
        [ 7.4601e-02, -1.4681e-01],
        [ 2.1566e-01, -2.3239e-01],
        [ 5.0469e-01, -1.1591e+00],
        [ 1.0509e-01, -2.8289e-01],
        [ 4.7329e-02, -5.1647e-01],
        [-3.4204e-02,  4.6150e-01],
        [ 1.0979e-01,  3.5336e-02],
        [ 3.9531e-01, -4.240

In [5]:
# Bytecode transformation
torch._logging.set_logs(graph=True)
torch._dynamo.reset()

print(opt_model)
opt_model(input_batch)

OptimizedModule(
  (_orig_mod): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), pad

[2023-09-07 14:55:37,264] [2/0] torch._dynamo.output_graph.__graph: [DEBUG] TRACED GRAPH
[2023-09-07 14:55:37,264] [2/0] torch._dynamo.output_graph.__graph: [DEBUG]  __compiled_fn_2 <eval_with_key>.175 opcode         name                                         target                                       args                                                                               kwargs
[2023-09-07 14:55:37,264] [2/0] torch._dynamo.output_graph.__graph: [DEBUG] -------------  -------------------------------------------  -------------------------------------------  ---------------------------------------------------------------------------------  --------
[2023-09-07 14:55:37,264] [2/0] torch._dynamo.output_graph.__graph: [DEBUG] placeholder    l_x_                                         L_x_                                         ()                                                                                 {}
[2023-09-07 14:55:37,264] [2/0] torch._dynamo.output_graph.__gr

tensor([[ 5.7302e-01,  1.6953e-01],
        [ 3.1415e-01, -1.9879e-01],
        [ 2.2843e-01, -5.1540e-02],
        [ 1.3476e-01, -5.3632e-01],
        [ 2.5758e-01,  3.6784e-01],
        [-2.0255e-01,  2.4212e-01],
        [-7.0895e-01,  4.0644e-02],
        [ 8.6189e-01, -1.9660e-01],
        [ 1.3331e-01, -2.1293e-01],
        [ 3.6175e-01,  1.1753e-01],
        [ 8.8030e-02, -7.1310e-01],
        [ 7.2991e-02,  1.0708e-01],
        [-1.9656e-02, -4.7510e-01],
        [-2.2431e-03,  2.8744e-01],
        [ 3.5546e-02, -4.8751e-01],
        [-5.6303e-01, -1.1960e-01],
        [ 5.1516e-01,  6.0561e-01],
        [-4.4132e-01,  1.2456e-01],
        [ 3.2252e-01,  1.1031e-03],
        [ 4.3625e-01, -1.1928e-01],
        [ 7.4601e-02, -1.4681e-01],
        [ 2.1566e-01, -2.3239e-01],
        [ 5.0469e-01, -1.1591e+00],
        [ 1.0509e-01, -2.8289e-01],
        [ 4.7329e-02, -5.1647e-01],
        [-3.4204e-02,  4.6150e-01],
        [ 1.0979e-01,  3.5335e-02],
        [ 3.9531e-01, -4.240

In [6]:
# Guards
torch._logging.set_logs(guards=True)
torch._dynamo.reset()
opt_model(input_batch)

[2023-09-07 14:56:40,182] [3/0] torch._dynamo.guards.__guards: [DEBUG] GUARDS:
[2023-09-07 14:56:40,184] [3/0] torch._dynamo.guards.__guards: [DEBUG] hasattr(L['x'], '_dynamo_dynamic_indices') == False           # _dynamo/variables/builder.py:1244 in wrap_fx_proxy_cls
[2023-09-07 14:56:40,184] [3/0] torch._dynamo.guards.__guards: [DEBUG] ___check_obj_id(L['self'], 139853266183168)                   # x = self.conv1(x)  # timm/models/resnet.py:520 in forward_features
[2023-09-07 14:56:40,185] [3/0] torch._dynamo.guards.__guards: [DEBUG] L['self'].training == True                                    # x = self.conv1(x)  # timm/models/resnet.py:520 in forward_features
[2023-09-07 14:56:40,185] [3/0] torch._dynamo.guards.__guards: [DEBUG] ___check_obj_id(L['self'].forward_head.__defaults__[0], 9474016)  # return x if pre_logits else self.fc(x)  # timm/models/resnet.py:538 in forward_head
[2023-09-07 14:56:40,185] [3/0] torch._dynamo.guards.__guards: [DEBUG] ___is_grad_enabled()             

tensor([[ 5.7302e-01,  1.6953e-01],
        [ 3.1415e-01, -1.9879e-01],
        [ 2.2843e-01, -5.1541e-02],
        [ 1.3476e-01, -5.3632e-01],
        [ 2.5758e-01,  3.6784e-01],
        [-2.0255e-01,  2.4212e-01],
        [-7.0895e-01,  4.0644e-02],
        [ 8.6189e-01, -1.9660e-01],
        [ 1.3331e-01, -2.1293e-01],
        [ 3.6175e-01,  1.1752e-01],
        [ 8.8030e-02, -7.1310e-01],
        [ 7.2991e-02,  1.0708e-01],
        [-1.9656e-02, -4.7510e-01],
        [-2.2432e-03,  2.8744e-01],
        [ 3.5546e-02, -4.8751e-01],
        [-5.6303e-01, -1.1960e-01],
        [ 5.1516e-01,  6.0561e-01],
        [-4.4132e-01,  1.2456e-01],
        [ 3.2252e-01,  1.1031e-03],
        [ 4.3625e-01, -1.1928e-01],
        [ 7.4601e-02, -1.4681e-01],
        [ 2.1566e-01, -2.3239e-01],
        [ 5.0469e-01, -1.1591e+00],
        [ 1.0509e-01, -2.8289e-01],
        [ 4.7329e-02, -5.1647e-01],
        [-3.4204e-02,  4.6150e-01],
        [ 1.0979e-01,  3.5335e-02],
        [ 3.9531e-01, -4.240

In [7]:
# Dynamo log
torch._logging.set_logs(dynamo=logging.DEBUG)
torch._dynamo.reset()
opt_model(input_batch)

[2023-09-07 14:57:21,920] [4/0] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing forward /usr/local/lib/python3.8/dist-packages/timm/models/resnet.py:540
[2023-09-07 14:57:21,922] [4/0] torch._dynamo.symbolic_convert.__trace_source: [DEBUG] TRACE starts_line forward /usr/local/lib/python3.8/dist-packages/timm/models/resnet.py:540
[2023-09-07 14:57:21,922] [4/0] torch._dynamo.symbolic_convert.__trace_source: [DEBUG]         def forward(self, x):
[2023-09-07 14:57:21,925] [4/0] torch._dynamo.variables.builder: [DEBUG] wrap_to_fake L['x'] (64, 3, 7, 7) [<DimDynamic.STATIC: 2>, <DimDynamic.STATIC: 2>, <DimDynamic.STATIC: 2>, <DimDynamic.STATIC: 2>] [None, None, None, None]
[2023-09-07 14:57:21,926] [4/0] torch._dynamo.symbolic_convert.__trace_source: [DEBUG] TRACE starts_line forward /usr/local/lib/python3.8/dist-packages/timm/models/resnet.py:541
[2023-09-07 14:57:21,926] [4/0] torch._dynamo.symbolic_convert.__trace_source: [DEBUG]             x = self.forward_feat

tensor([[ 5.7302e-01,  1.6953e-01],
        [ 3.1415e-01, -1.9879e-01],
        [ 2.2843e-01, -5.1540e-02],
        [ 1.3476e-01, -5.3632e-01],
        [ 2.5758e-01,  3.6784e-01],
        [-2.0255e-01,  2.4212e-01],
        [-7.0895e-01,  4.0644e-02],
        [ 8.6189e-01, -1.9660e-01],
        [ 1.3331e-01, -2.1293e-01],
        [ 3.6175e-01,  1.1752e-01],
        [ 8.8030e-02, -7.1310e-01],
        [ 7.2991e-02,  1.0708e-01],
        [-1.9656e-02, -4.7510e-01],
        [-2.2430e-03,  2.8744e-01],
        [ 3.5546e-02, -4.8751e-01],
        [-5.6303e-01, -1.1960e-01],
        [ 5.1516e-01,  6.0561e-01],
        [-4.4132e-01,  1.2456e-01],
        [ 3.2252e-01,  1.1028e-03],
        [ 4.3625e-01, -1.1928e-01],
        [ 7.4601e-02, -1.4681e-01],
        [ 2.1566e-01, -2.3239e-01],
        [ 5.0469e-01, -1.1591e+00],
        [ 1.0509e-01, -2.8289e-01],
        [ 4.7329e-02, -5.1647e-01],
        [-3.4204e-02,  4.6150e-01],
        [ 1.0979e-01,  3.5335e-02],
        [ 3.9531e-01, -4.240

### AOTAutograd

In [8]:
# AOTAutograd Lowering - forwards & backwards pass
torch._logging.set_logs(graph_code=True, aot_graphs=True)
torch._dynamo.reset()
opt_model(input_batch)

[2023-09-07 14:58:47,082] [5/0] torch._dynamo.output_graph.__graph_code: [DEBUG] TRACED GRAPH
[2023-09-07 14:58:47,082] [5/0] torch._dynamo.output_graph.__graph_code: [DEBUG]  ===== __compiled_fn_5 =====
[2023-09-07 14:58:47,082] [5/0] torch._dynamo.output_graph.__graph_code: [DEBUG]  <eval_with_key>.220 class GraphModule(torch.nn.Module):
[2023-09-07 14:58:47,082] [5/0] torch._dynamo.output_graph.__graph_code: [DEBUG]     def forward(self, L_x_ : torch.Tensor):
[2023-09-07 14:58:47,082] [5/0] torch._dynamo.output_graph.__graph_code: [DEBUG]         l_x_ = L_x_
[2023-09-07 14:58:47,082] [5/0] torch._dynamo.output_graph.__graph_code: [DEBUG]         
[2023-09-07 14:58:47,082] [5/0] torch._dynamo.output_graph.__graph_code: [DEBUG]         # File: /usr/local/lib/python3.8/dist-packages/timm/models/resnet.py:520, code: x = self.conv1(x)
[2023-09-07 14:58:47,082] [5/0] torch._dynamo.output_graph.__graph_code: [DEBUG]         l__self___conv1 = self.L__self___conv1(l_x_);  l_x_ = None
[2023-0

tensor([[ 5.7302e-01,  1.6953e-01],
        [ 3.1415e-01, -1.9879e-01],
        [ 2.2843e-01, -5.1540e-02],
        [ 1.3476e-01, -5.3632e-01],
        [ 2.5758e-01,  3.6784e-01],
        [-2.0255e-01,  2.4212e-01],
        [-7.0895e-01,  4.0644e-02],
        [ 8.6189e-01, -1.9660e-01],
        [ 1.3331e-01, -2.1293e-01],
        [ 3.6175e-01,  1.1752e-01],
        [ 8.8030e-02, -7.1310e-01],
        [ 7.2991e-02,  1.0708e-01],
        [-1.9656e-02, -4.7510e-01],
        [-2.2432e-03,  2.8744e-01],
        [ 3.5546e-02, -4.8751e-01],
        [-5.6303e-01, -1.1960e-01],
        [ 5.1516e-01,  6.0561e-01],
        [-4.4132e-01,  1.2456e-01],
        [ 3.2252e-01,  1.1030e-03],
        [ 4.3625e-01, -1.1928e-01],
        [ 7.4601e-02, -1.4681e-01],
        [ 2.1566e-01, -2.3239e-01],
        [ 5.0469e-01, -1.1591e+00],
        [ 1.0509e-01, -2.8289e-01],
        [ 4.7329e-02, -5.1647e-01],
        [-3.4204e-02,  4.6150e-01],
        [ 1.0979e-01,  3.5335e-02],
        [ 3.9531e-01, -4.240

### Inductor

In [9]:
torch._logging.set_logs(inductor=logging.DEBUG)
torch._dynamo.reset()
opt_model(input_batch)

[2023-09-07 15:00:05,880] [6/0] torch._inductor.compile_fx: [INFO] Step 3: torchinductor compiling FORWARDS graph 6
[2023-09-07 15:00:07,022] [6/0] torch._inductor.graph: [DEBUG] Force channels last inputs for 20 conv for the current graph with id 6
[2023-09-07 15:00:07,670] [6/0] torch._inductor.scheduler: [DEBUG] removed dead node: buf24
[2023-09-07 15:00:07,670] [6/0] torch._inductor.scheduler: [DEBUG] removed dead node: buf37
[2023-09-07 15:00:07,671] [6/0] torch._inductor.scheduler: [DEBUG] removed dead node: buf48
[2023-09-07 15:00:07,671] [6/0] torch._inductor.scheduler: [DEBUG] removed dead node: buf59
[2023-09-07 15:00:07,671] [6/0] torch._inductor.scheduler: [DEBUG] removed dead node: buf70
[2023-09-07 15:00:07,672] [6/0] torch._inductor.scheduler: [DEBUG] removed dead node: buf78
[2023-09-07 15:00:07,672] [6/0] torch._inductor.scheduler: [DEBUG] removed dead node: buf86
[2023-09-07 15:00:07,672] [6/0] torch._inductor.scheduler: [DEBUG] removed dead node: buf93
[2023-09-07 15

tensor([[ 5.7302e-01,  1.6953e-01],
        [ 3.1415e-01, -1.9879e-01],
        [ 2.2843e-01, -5.1540e-02],
        [ 1.3476e-01, -5.3632e-01],
        [ 2.5758e-01,  3.6784e-01],
        [-2.0255e-01,  2.4212e-01],
        [-7.0895e-01,  4.0644e-02],
        [ 8.6189e-01, -1.9660e-01],
        [ 1.3331e-01, -2.1293e-01],
        [ 3.6175e-01,  1.1752e-01],
        [ 8.8030e-02, -7.1310e-01],
        [ 7.2991e-02,  1.0708e-01],
        [-1.9656e-02, -4.7510e-01],
        [-2.2432e-03,  2.8744e-01],
        [ 3.5546e-02, -4.8751e-01],
        [-5.6303e-01, -1.1960e-01],
        [ 5.1516e-01,  6.0561e-01],
        [-4.4132e-01,  1.2456e-01],
        [ 3.2252e-01,  1.1030e-03],
        [ 4.3625e-01, -1.1928e-01],
        [ 7.4601e-02, -1.4681e-01],
        [ 2.1566e-01, -2.3239e-01],
        [ 5.0469e-01, -1.1591e+00],
        [ 1.0509e-01, -2.8289e-01],
        [ 4.7329e-02, -5.1647e-01],
        [-3.4204e-02,  4.6150e-01],
        [ 1.0979e-01,  3.5335e-02],
        [ 3.9531e-01, -4.240

In [10]:
torch._logging.set_logs(output_code=True)
torch._dynamo.reset()
opt_model(input_batch)

[2023-09-07 15:00:49,059] [7/0] torch._inductor.graph.__output_code: [INFO] Output code written to: /tmp/torchinductor_root/iq/ciqegoc3cigmvkqeaytmfiekrtinzr4hgvnrgl4fhnlec5v75iha.py
[2023-09-07 15:00:49,060] [7/0] torch._inductor.graph.__output_code: [DEBUG] Output code: 
[2023-09-07 15:00:49,060] [7/0] torch._inductor.graph.__output_code: [DEBUG] 
[2023-09-07 15:00:49,060] [7/0] torch._inductor.graph.__output_code: [DEBUG] from ctypes import c_void_p, c_long
[2023-09-07 15:00:49,060] [7/0] torch._inductor.graph.__output_code: [DEBUG] import torch
[2023-09-07 15:00:49,060] [7/0] torch._inductor.graph.__output_code: [DEBUG] import math
[2023-09-07 15:00:49,060] [7/0] torch._inductor.graph.__output_code: [DEBUG] import random
[2023-09-07 15:00:49,060] [7/0] torch._inductor.graph.__output_code: [DEBUG] import os
[2023-09-07 15:00:49,060] [7/0] torch._inductor.graph.__output_code: [DEBUG] import tempfile
[2023-09-07 15:00:49,060] [7/0] torch._inductor.graph.__output_code: [DEBUG] from mat

tensor([[ 5.7302e-01,  1.6953e-01],
        [ 3.1415e-01, -1.9879e-01],
        [ 2.2843e-01, -5.1540e-02],
        [ 1.3476e-01, -5.3632e-01],
        [ 2.5758e-01,  3.6784e-01],
        [-2.0255e-01,  2.4212e-01],
        [-7.0895e-01,  4.0644e-02],
        [ 8.6189e-01, -1.9660e-01],
        [ 1.3331e-01, -2.1293e-01],
        [ 3.6175e-01,  1.1752e-01],
        [ 8.8030e-02, -7.1310e-01],
        [ 7.2991e-02,  1.0708e-01],
        [-1.9656e-02, -4.7510e-01],
        [-2.2430e-03,  2.8744e-01],
        [ 3.5546e-02, -4.8751e-01],
        [-5.6303e-01, -1.1960e-01],
        [ 5.1516e-01,  6.0561e-01],
        [-4.4132e-01,  1.2456e-01],
        [ 3.2252e-01,  1.1030e-03],
        [ 4.3625e-01, -1.1928e-01],
        [ 7.4601e-02, -1.4681e-01],
        [ 2.1566e-01, -2.3239e-01],
        [ 5.0469e-01, -1.1591e+00],
        [ 1.0509e-01, -2.8289e-01],
        [ 4.7329e-02, -5.1647e-01],
        [-3.4204e-02,  4.6150e-01],
        [ 1.0979e-01,  3.5335e-02],
        [ 3.9531e-01, -4.240

## PT2.0 Profiling

### Regular torch.profile()
Warmup helps clean up the trace

In [13]:
import torch
from torchvision.models import resnet18

torch._logging.set_logs()
torch._dynamo.reset()

model = resnet18().cuda()
inputs = [torch.randn((5, 3, 224, 224), device='cuda') for _ in range(10)]

model_c = torch.compile(model)

def fwd_bwd(inp):
    out = model_c(inp)
    out.sum().backward()

# warm up
fwd_bwd(inputs[0])

with torch.profiler.profile() as prof:
    for i in range(1, 4):
        fwd_bwd(inputs[i])
        prof.step()

prof.export_chrome_trace("cold.json")

STAGE:2023-09-07 15:29:36 988556:988556 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-09-07 15:29:44 988556:988556 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-09-07 15:29:44 988556:988556 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


### Understanding compilation time

In [14]:
import torch
from torchvision.models import resnet18

torch._logging.set_logs()
torch._dynamo.reset()

model = resnet18().cuda()
inputs = [torch.randn((5, 3, 224, 224), device='cuda') for _ in range(10)]

model_c = torch.compile(model)

def fwd_bwd(inp):
    out = model_c(inp)
    out.sum().backward()

with torch.profiler.profile() as prof:
    with torch.profiler.record_function("resnet18 compile"):
        fwd_bwd(inputs[0])

prof.export_chrome_trace("trace_compile.json")

print(torch._dynamo.utils.compile_times())

STAGE:2023-09-07 16:19:26 988556:988556 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-09-07 16:19:35 988556:988556 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-09-07 16:19:35 988556:988556 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


TorchDynamo compilation metrics:
Function                              Runtimes (s)
------------------------------------  --------------
_compile.<locals>.compile_inner       6.1978
OutputGraph.call_user_compiler        5.8156
create_aot_dispatcher_function        5.8038
compile_fx.<locals>.fw_compiler_base  3.9522
GraphLowering.run                     0.6129, 0.6465
GraphLowering.compile_to_module       2.7241, 1.0656
Scheduler.__init__                    1.7208, 0.5109
Scheduler.codegen                     0.9591, 0.5100
WrapperCodeGen.generate               0.0371, 0.0307
compile_fx.<locals>.bw_compiler       2.1654


In [15]:
!pip install transformers

In [9]:
import transformers 
from transformers import BertTokenizer, BertModel

torch._logging.set_logs()
torch._dynamo.reset()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased").to(device="cuda:0")
model = torch.compile(model, backend="inductor", dynamic=False, fullgraph=False) # This is the only line of code that we changed

text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt').to(device="cuda:0")
output = model(**encoded_input)

print(torch._dynamo.utils.compile_times())

TorchDynamo compilation metrics:
Function                                Runtimes (s)
--------------------------------------  -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
_compile.<locals>.compile_inner         15.2372
OutputGraph.call_user_compiler          13.0239
create_aot_dispatcher_function          0.0161, 0.0159, 0.0154, 0.0155, 0.0156, 0.0290, 0.0159, 0.0168, 0.0156, 0.0155, 0.0156, 0.0156, 0.0160, 0.0165, 0.0155, 0.0156, 0.0156, 0.0156, 0.0157, 0.0163, 0.0156, 0.0155, 0.0155, 0.0154, 12.9635
compile_fx.<locals>.fw_compiler_base    5.3141
GraphLowering.run                       0.7436
GraphLowering.compile_to_module         3.2785
Scheduler.__init__                      0.9226
Scheduler.codegen                       0.5623
WrapperCodeGen.generate                 0.0579
CachingAutotuner.benchmark_all_configs  0.1817, 0.0926, 

In [10]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt').to(device="cuda:0")
output = model(**encoded_input)

In [30]:
torch._logging.set_logs(recompiles=True, graph_breaks=True)

text = "Testing purposes"
encoded_input = tokenizer(text, return_tensors='pt').to(device="cuda:0")
output = model(**encoded_input)

[2023-09-07 16:35:12,266] torch._dynamo.convert_frame.__recompiles: [DEBUG] ('Recompiling function forward in /usr/local/lib/python3.8/dist-packages/transformers/models/bert/modeling_bert.py:913', "triggered by the following guard failure: tensor 'L['input_ids']' stride mismatch at index 0. expected 12, actual 4")


In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Set the logging configurations
torch._logging.set_logs(recompiles=True)

def get_predictions(text_list):
    """
    Given a list of strings, tokenize and pass them to the model for predictions.
    :param text_list: List of strings
    :return: List of model outputs
    """
    outputs = []
    for text in text_list:
        encoded_input = tokenizer(text, return_tensors='pt').to(device="cuda:0")
        output = model(**encoded_input)
        outputs.append(output)
    return outputs

# Your training set of 10 strings
training_set = [
    "Hello world!",
    "How are you?",
    "What's the weather like?",
    "I love reading books.",
    "The movie was fantastic!",
    "My cat is adorable.",
    "Let's go for a hike.",
    "Coffee is my morning fuel.",
    "The concert was a blast!",
    "Keep calm and code on."
]

# Fetch predictions for the training set
predictions = get_predictions(training_set)
print(predictions)


[2023-09-07 16:36:22,141] torch._dynamo.convert_frame.__recompiles: [DEBUG] ('Recompiling function forward in /usr/local/lib/python3.8/dist-packages/transformers/models/bert/modeling_bert.py:913', "triggered by the following guard failure: tensor 'L['input_ids']' stride mismatch at index 0. expected 12, actual 5")
[2023-09-07 16:36:36,570] torch._dynamo.convert_frame.__recompiles: [DEBUG] ('Recompiling function forward in /usr/local/lib/python3.8/dist-packages/transformers/models/bert/modeling_bert.py:913', "triggered by the following guard failure: tensor 'L['input_ids']' stride mismatch at index 0. expected 5, actual 6")
[2023-09-07 16:36:51,075] torch._dynamo.convert_frame.__recompiles: [DEBUG] ('Recompiling function forward in /usr/local/lib/python3.8/dist-packages/transformers/models/bert/modeling_bert.py:913', "triggered by the following guard failure: tensor 'L['input_ids']' stride mismatch at index 0. expected 6, actual 9")
[2023-09-07 16:37:07,950] torch._dynamo.convert_frame.

[BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1424,  0.1335, -0.1291,  ..., -0.3597, -0.0562,  0.3605],
         [-0.3506,  0.1042,  0.6244,  ..., -0.1761,  0.4834,  0.0644],
         [-0.2451, -0.1573,  0.6945,  ..., -0.5654, -0.0894, -0.1856],
         [-0.8248, -0.9119, -0.6561,  ...,  0.5074, -0.1939, -0.1659],
         [ 0.8767,  0.0352, -0.1233,  ...,  0.2720, -0.6369, -0.1585]]],
       device='cuda:0', grad_fn=<CompiledFunctionBackward>), pooler_output=tensor([[-8.9756e-01, -3.3040e-01, -7.6942e-01,  7.5799e-01,  4.6678e-01,
         -1.2035e-01,  9.1835e-01,  1.8087e-01, -7.2716e-01, -9.9991e-01,
         -4.4723e-01,  8.9104e-01,  9.6621e-01,  5.4915e-01,  9.4344e-01,
         -7.6605e-01, -6.0469e-01, -6.1654e-01,  4.0572e-01, -7.4644e-01,
          6.1739e-01,  9.9974e-01,  3.2991e-02,  2.5414e-01,  4.3106e-01,
          9.7732e-01, -8.4328e-01,  9.2297e-01,  9.4871e-01,  6.3994e-01,
         -7.3620e-01,  9.0956e-02, -9.7607e-01, -1.9115e-01,

In [34]:
from torch._dynamo.utils import CompileProfiler

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Set the logging configurations
torch._logging.set_logs(recompiles=True, graph_breaks=True)

def get_predictions(text_list):
    """
    Given a list of strings, tokenize and pass them to the model for predictions.
    :param text_list: List of strings
    :return: List of model outputs
    """
    outputs = []
    for text in text_list:
        encoded_input = tokenizer(text, return_tensors='pt').to(device="cuda:0")
        output = model(**encoded_input)
        outputs.append(output)
    return outputs

# Your training set of 10 strings
training_set = [
    "Hello world!",
    "How are you?",
    "What's the weather like?",
    "I love reading books.",
    "The movie was fantastic!",
    "My cat is adorable.",
    "Let's go for a hike.",
    "Coffee is my morning fuel.",
    "The concert was a blast!",
    "Keep calm and code on."
]

# Fetch predictions for the training set
with CompileProfiler() as prof:
    predictions = get_predictions(training_set)
    print(prof.report())

print(torch._dynamo.utils.compile_times())



[2023-09-07 16:38:31,394] torch._dynamo.convert_frame.__recompiles: [DEBUG] ('Recompiling function forward in /usr/local/lib/python3.8/dist-packages/transformers/models/bert/modeling_bert.py:913', "triggered by the following guard failure: tensor 'L['input_ids']' stride mismatch at index 0. expected 12, actual 5")
[2023-09-07 16:38:44,209] torch._dynamo.convert_frame.__recompiles: [DEBUG] ('Recompiling function forward in /usr/local/lib/python3.8/dist-packages/transformers/models/bert/modeling_bert.py:913', "triggered by the following guard failure: tensor 'L['input_ids']' stride mismatch at index 0. expected 5, actual 6")
[2023-09-07 16:38:59,636] torch._dynamo.convert_frame.__recompiles: [DEBUG] ('Recompiling function forward in /usr/local/lib/python3.8/dist-packages/transformers/models/bert/modeling_bert.py:913', "triggered by the following guard failure: tensor 'L['input_ids']' stride mismatch at index 0. expected 6, actual 9")
[2023-09-07 16:39:13,068] torch._dynamo.convert_frame.


Torchdynamo Profiler Report

Graph Breaks
------------
Graph breaks happen when torchdynamo encounters code it can't safely trace.
If you want to find out why breaks are happening, check below for each break reason
You may gain additional insight by passing `fullgraph=True` to torch.compile,
to stop at the first break.

No graph breaks detected.

Recompilation
-------------
These subgraphs were recompiled more than once due to guard failures
Guard failures indicate some condition assumed to be static by the tracer changed,
making it unsafe to reuse the compiled program.

Function                            Recompiles  Recompile Reasons
--------------------------------  ------------  -------------------------------------------------------------------------
'forward' (modeling_bert.py:913)             6  tensor 'L['input_ids']' stride mismatch at index 0. expected 12, actual 5
                                                tensor 'L['input_ids']' stride mismatch at index 0. expected 5,

### Perf optimisation - Dynamic shapes 

In [88]:
import transformers 
from transformers import BertTokenizer, BertModel

torch._logging.set_logs()
torch._dynamo.reset()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased").to(device="cuda:0")
model = torch.compile(model, backend="inductor", dynamic=False) # This is the only line of code that we changed

text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt').to(device="cuda:0")
output = model(**encoded_input)

# Set the logging configurations
torch._logging.set_logs(recompiles=True, graph_breaks=True)

def get_predictions(text_list):
    """
    Given a list of strings, tokenize and pass them to the model for predictions.
    :param text_list: List of strings
    :return: List of model outputs
    """
    outputs = []
    for text in text_list:
        encoded_input = tokenizer(text, return_tensors='pt').to(device="cuda:0")
        output = model(**encoded_input)
        outputs.append(output)
    return outputs

# Your training set of 10 strings
training_set = [
    "Hello world!",
    "How are you?",
    "What's the weather like?",
    "I love reading books.",
    "The movie was fantastic!",
    "My cat is adorable.",
    "Let's go for a hike.",
    "Coffee is my morning fuel.",
    "The concert was a blast!",
    "Keep calm and code on."
]

# Fetch predictions for the training set
with CompileProfiler() as prof:
    predictions = get_predictions(training_set)
    print(prof.report())

print(torch._dynamo.utils.compile_times())


[2023-09-07 01:34:26,263] torch._dynamo.convert_frame.__recompiles: [DEBUG] ('Recompiling function forward in /usr/local/lib/python3.8/dist-packages/transformers/models/bert/modeling_bert.py:913', "triggered by the following guard failure: tensor 'L['input_ids']' stride mismatch at index 0. expected 12, actual 5")
[2023-09-07 01:34:38,597] torch._dynamo.convert_frame.__recompiles: [DEBUG] ('Recompiling function forward in /usr/local/lib/python3.8/dist-packages/transformers/models/bert/modeling_bert.py:913', "triggered by the following guard failure: tensor 'L['input_ids']' stride mismatch at index 0. expected 5, actual 6")
[2023-09-07 01:34:53,022] torch._dynamo.convert_frame.__recompiles: [DEBUG] ('Recompiling function forward in /usr/local/lib/python3.8/dist-packages/transformers/models/bert/modeling_bert.py:913', "triggered by the following guard failure: tensor 'L['input_ids']' stride mismatch at index 0. expected 6, actual 9")
[2023-09-07 01:35:08,995] torch._dynamo.convert_frame.


Torchdynamo Profiler Report

Graph Breaks
------------
Graph breaks happen when torchdynamo encounters code it can't safely trace.
If you want to find out why breaks are happening, check below for each break reason
You may gain additional insight by passing `fullgraph=True` to torch.compile,
to stop at the first break.

No graph breaks detected.

Recompilation
-------------
These subgraphs were recompiled more than once due to guard failures
Guard failures indicate some condition assumed to be static by the tracer changed,
making it unsafe to reuse the compiled program.

Function                            Recompiles  Recompile Reasons
--------------------------------  ------------  -------------------------------------------------------------------------
'forward' (modeling_bert.py:913)             6  tensor 'L['input_ids']' stride mismatch at index 0. expected 12, actual 5
                                                tensor 'L['input_ids']' stride mismatch at index 0. expected 5,

In [87]:
import transformers 
from transformers import BertTokenizer, BertModel

torch._logging.set_logs()
torch._dynamo.reset()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased").to(device="cuda:0")
model = torch.compile(model, backend="inductor", dynamic=True) # This is the only line of code that we changed

text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt').to(device="cuda:0")
output = model(**encoded_input)

# Set the logging configurations
torch._logging.set_logs(recompiles=True, graph_breaks=True)

def get_predictions(text_list):
    """
    Given a list of strings, tokenize and pass them to the model for predictions.
    :param text_list: List of strings
    :return: List of model outputs
    """
    outputs = []
    for text in text_list:
        encoded_input = tokenizer(text, return_tensors='pt').to(device="cuda:0")
        output = model(**encoded_input)
        outputs.append(output)
    return outputs

# Your training set of 10 strings
training_set = [
    "Hello world!",
    "How are you?",
    "What's the weather like?",
    "I love reading books.",
    "The movie was fantastic!",
    "My cat is adorable.",
    "Let's go for a hike.",
    "Coffee is my morning fuel.",
    "The concert was a blast!",
    "Keep calm and code on."
]

# Fetch predictions for the training set
with CompileProfiler() as prof:
    predictions = get_predictions(training_set)
    print(prof.report())

print(torch._dynamo.utils.compile_times())



Torchdynamo Profiler Report

Graph Breaks
------------
Graph breaks happen when torchdynamo encounters code it can't safely trace.
If you want to find out why breaks are happening, check below for each break reason
You may gain additional insight by passing `fullgraph=True` to torch.compile,
to stop at the first break.

No graph breaks detected.

Recompilation
-------------
These subgraphs were recompiled more than once due to guard failures
Guard failures indicate some condition assumed to be static by the tracer changed,
making it unsafe to reuse the compiled program.

No recompilation detected.

TorchDynamo compilation metrics:
Function                              Runtimes (s)
------------------------------------  -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
_compile.<locals>.compile_inner       28.9943
OutputGraph.call_user_com

### Minimising graph breaks

In [92]:
import torch
import torch._dynamo as dynamo
def toy_example(a, b):
    x = a / (torch.abs(a) + 1)
    print("woo")
    if b.sum() < 0:
        b = b * -1
    return x * b

explain = dynamo.explain(toy_example)(torch.randn(10), torch.randn(10))

print(explain.break_reasons)

[2023-09-07 01:39:32,189] [64/0] torch._dynamo.symbolic_convert.__graph_breaks: [DEBUG] Graph break: call_function BuiltinVariable(print) [ConstantVariable(str)] {} from user code at:
[2023-09-07 01:39:32,189] [64/0] torch._dynamo.symbolic_convert.__graph_breaks: [DEBUG]   File "/tmp/ipykernel_19834/3151414406.py", line 5, in toy_example
[2023-09-07 01:39:32,189] [64/0] torch._dynamo.symbolic_convert.__graph_breaks: [DEBUG]     print("woo")
[2023-09-07 01:39:32,189] [64/0] torch._dynamo.symbolic_convert.__graph_breaks: [DEBUG] 


woo
[GraphCompileReason(reason='call_function BuiltinVariable(print) [ConstantVariable(str)] {}', user_stack=[<FrameSummary file /tmp/ipykernel_19834/3151414406.py, line 5 in toy_example>], graph_break=True), GraphCompileReason(reason='generic_jump TensorVariable()', user_stack=[<FrameSummary file /tmp/ipykernel_19834/3151414406.py, line 6 in <resume in toy_example>>], graph_break=True)]


### CudaGraphs

In [6]:
import transformers 
from transformers import BertTokenizer, BertModel, T5Tokenizer, T5ForConditionalGeneration
import time
import torch

torch._logging.set_logs()
torch._dynamo.reset()

training_set = [
    "Each culture offers a unique flavor to the world.",
    "Time, an ever-flowing river, waits for none.",
    "Childhood memories are treasures of the heart.",
    "Love is the force that binds the universe.",
    "Dreams are where imagination knows no bounds.",
    "Laughter is the shortest distance between two people.",
    "Food is more than sustenance; it's an experience.",
    "Moonlit nights have a tale of their own.",
]

def get_predictions(model_instance, text_list):
    """
    Given a list of strings and a model instance, tokenize and pass them to the model for predictions.
    :param model_instance: Instance of the model to run predictions
    :param text_list: List of strings
    :return: List of model outputs
    """
    for text in text_list:
        encoded_input = tokenizer(text, return_tensors='pt').to(device="cuda:0")
        output = model_instance(**encoded_input)

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertModel.from_pretrained("bert-large-uncased").to(device="cuda:0")

real_eager = model
aot_eager = torch.compile(model, backend="aot_eager", dynamic=True)
model_ind = torch.compile(model, backend="inductor", dynamic=False)
model_dynamic_shapes = torch.compile(model, backend="inductor", dynamic=True)
model_dynamic_shapes_cuda_graph = torch.compile(model, backend="inductor", dynamic=True, mode="reduce-overhead")
model_autotune = torch.compile(model, backend="inductor", dynamic=True, mode="max-autotune")
model_autotune_nograph = torch.compile(model, backend="inductor", dynamic=True, mode="max-autotune-no-cudagraphs")

text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt').to(device="cuda:0")

#start_time = time.time()
#output = model(**encoded_input)
#elapsed_time = time.time() - start_time
#print(f"Eager Model took {elapsed_time:.4f} seconds")

models = [
    ("Real eager", real_eager),
    ("Eager compile", aot_eager),
    ("Inductor", model_ind),
    ("Dynamic Shapes", model_dynamic_shapes),
    ("Dynamic Shapes with CUDA Graph", model_dynamic_shapes_cuda_graph),
    ("Autotune", model_autotune),
    ("Autotune no cudagraph", model_autotune_nograph)
]

for name, model_instance in models:

    # Reset for actual timing
    torch._dynamo.reset()

    # Warmup
    model_instance(**tokenizer(training_set[0], return_tensors='pt').to(device="cuda:0"))

    start_time = time.time()
    get_predictions(model_instance, training_set)
    elapsed_time = time.time() - start_time
    torch.cuda.synchronize()

    print(f"{name} (warmup) Model took {elapsed_time:.4f} seconds")

    start_time = time.time()
    get_predictions(model_instance, training_set)
    elapsed_time = time.time() - start_time
    torch.cuda.synchronize()
    
    print(f"{name} Model took {elapsed_time:.4f} seconds")

    
print(torch._dynamo.utils.compile_times())


Real eager (warmup) Model took 0.1197 seconds
Real eager Model took 0.1192 seconds
Eager compile (warmup) Model took 0.3551 seconds
Eager compile Model took 0.3525 seconds


/usr/local/lib/python3.8/dist-packages/torch/_inductor/compile_fx.py:135: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


Inductor (warmup) Model took 111.2118 seconds
Inductor Model took 0.0899 seconds
Dynamic Shapes (warmup) Model took 0.0908 seconds
Dynamic Shapes Model took 0.0898 seconds
Dynamic Shapes with CUDA Graph (warmup) Model took 0.8104 seconds
Dynamic Shapes with CUDA Graph Model took 8.6877 seconds


AUTOTUNE addmm(12x1024, 12x1024, 1024x1024)
  bias_addmm 0.0328 ms 100.0%
  addmm 0.0330 ms 99.5%
  triton_mm_3 0.1293 ms 25.4%
  triton_mm_6 0.1520 ms 21.6%
  triton_mm_4 0.3574 ms 9.2%
  triton_mm_0 0.3732 ms 8.8%
  triton_mm_5 0.3848 ms 8.5%
  triton_mm_1 0.3960 ms 8.3%
  triton_mm_2 0.7030 ms 4.7%
SingleProcess AUTOTUNE takes 4.4057 seconds
AUTOTUNE bmm(16x12x64, 16x64x12)
  bmm 0.0146 ms 100.0%
  triton_bmm_22 0.0160 ms 91.0%
  triton_bmm_21 0.0170 ms 85.8%
  triton_bmm_23 0.0189 ms 77.1%
SingleProcess AUTOTUNE takes 2.0906 seconds
AUTOTUNE bmm(16x12x12, 16x12x64)
  bmm 0.0144 ms 100.0%
  triton_bmm_25 0.0147 ms 97.8%
  triton_bmm_24 0.0149 ms 96.8%
SingleProcess AUTOTUNE takes 1.0130 seconds
AUTOTUNE addmm(12x4096, 12x1024, 1024x4096)
  bias_addmm 0.0523 ms 100.0%
  addmm 0.0523 ms 100.0%
  triton_mm_36 0.2053 ms 25.5%
  triton_mm_39 0.2078 ms 25.2%
  triton_mm_37 0.3613 ms 14.5%
  triton_mm_33 0.3736 ms 14.0%
  triton_mm_38 0.3885 ms 13.5%
  triton_mm_34 0.3910 ms 13.4%
  triton

Autotune (warmup) Model took 0.8872 seconds
Autotune Model took 9.0501 seconds
Autotune no cudagraph (warmup) Model took 0.1079 seconds
Autotune no cudagraph Model took 0.1071 seconds
TorchDynamo compilation metrics:
Function                              Runtimes (s)
------------------------------------  -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
_compile.<locals>.compile_inner       66.3536
OutputGraph.call_user_compiler        58.7143
create_aot_dispatcher_function        0.0356, 0.0344, 0.0355, 0.0341, 0.0362, 0.0352, 0.0354, 0.0350, 0.0361, 0.0352, 0.0370, 0.0343, 0.0355, 0.0356, 0.0360, 0.0351, 0.0360, 0.0351, 0.0362, 0.0352, 0.0366, 0.03